In [54]:
import chardet
import numpy as np
import scipy as sp
import scipy.constants as constants
import pandas as pd

R = 0.20
mu_0 = constants.mu_0

print(mu_0)

n = 154.0

raios = [2, 3, 4, 5]

amp = []
ampPerRadius = []

volt = []
voltPerRadius = []

magneticField = []    
magneticFieldPerRadius = []

for raio in raios:
    with open(f'data/raio{raio}cm.txt', 'rb') as f:
        enc = chardet.detect(f.read())  # or readline if the file is large

    file = pd.read_csv(f'data/raio{raio}cm.txt', header = None, sep='\s+', encoding=enc['encoding'], dtype = np.float64)
    
    for data in [file[0]]:
        amp.append(data)
        B = np.sqrt((4.0/5.0)*(4.0/5.0)*(4.0/5.0)) * ((mu_0 * n * (data))/R)
        magneticField.append(B)
        
    for data in [file[1]]:
        volt.append(data)
        
    magneticFieldPerRadius.append(magneticField)
    ampPerRadius.append(amp)
    voltPerRadius.append(volt)
    
    amp = []
    volt = []
    magneticField = []

print(ampPerRadius)
print(voltPerRadius)
print(magneticFieldPerRadius)

1.25663706212e-06
[[0    2.50
1    2.76
2    3.06
3    3.26
4    3.33
5    3.60
6    3.96
7    4.00
8    4.08
Name: 0, dtype: float64], [0     1.96
1     2.05
2     2.14
3     2.28
4     2.34
5     2.46
6     2.58
7     2.69
8     2.81
9     2.90
10    3.00
11    3.90
12    3.13
Name: 0, dtype: float64], [0     1.50
1     1.63
2     1.68
3     1.74
4     1.83
5     1.91
6     2.00
7     2.07
8     2.13
9     2.21
10    2.29
11    2.31
Name: 0, dtype: float64], [0     1.00
1     1.13
2     1.24
3     1.31
4     1.38
5     1.45
6     1.53
7     1.59
8     1.66
9     1.71
10    1.78
11    1.82
12    1.85
Name: 0, dtype: float64]]
[[0    105.0
1    120.0
2    140.0
3    160.0
4    180.0
5    200.0
6    240.0
7    246.0
8    260.0
Name: 1, dtype: float64], [0     140.0
1     150.0
2     170.0
3     190.0
4     200.0
5     220.0
6     240.0
7     260.0
8     280.0
9     300.0
10    320.0
11    340.0
12    350.0
Name: 1, dtype: float64], [0     150.0
1     170.0
2     190.0
3     200.0
4     

In [55]:
raios = [0.02, 0.03, 0.04, 0.05]

chargePerMassPerRadius = []
chargePerMass = []

for i in range(np.size(voltPerRadius)):
    j = 0
    for volt in voltPerRadius[i]:
        CpM = 2.0*(volt)/(raios[i]*raios[i]*(magneticFieldPerRadius[i][j])*(magneticFieldPerRadius[i][j]))
        chargePerMass.append(CpM)
        j = j + 1
        
    chargePerMassPerRadius.append(chargePerMass)
    chargePerMass = []
    
print(chargePerMassPerRadius)

[[0    1.752299e+11
1    1.643092e+11
2    1.559495e+11
3    1.570304e+11
4    1.693101e+11
5    1.609622e+11
6    1.596319e+11
7    1.603666e+11
8    1.629116e+11
dtype: float64], [0     1.689399e+11
1     1.654626e+11
2     1.720829e+11
3     1.694339e+11
4     1.693225e+11
5     1.685267e+11
6     1.671430e+11
7     1.665655e+11
8     1.643848e+11
9     1.653642e+11
10    1.648253e+11
11    1.036254e+11
12    1.656135e+11
dtype: float64], [0     1.738392e+11
1     1.668448e+11
2     1.755391e+11
3     1.722544e+11
4     1.713008e+11
5     1.715471e+11
6     1.694932e+11
7     1.703948e+11
8     1.724253e+11
9     1.708458e+11
10    1.690623e+11
11    1.715229e+11
dtype: float64], [0     2.002627e+11
1     1.829743e+11
2     1.736583e+11
3     1.750446e+11
4     1.752632e+11
5     1.738309e+11
6     1.710989e+11
7     1.716319e+11
8     1.695746e+11
9     1.712174e+11
10    1.685500e+11
11    1.718029e+11
12    1.711522e+11
dtype: float64]]


c:\Python310\lib\site-packages\numpy\core\fromnumeric.py:3245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


In [56]:
tab = '\t'

raio = []
volts = []
amps = []
mF = []
CpM = []

for i in range(np.size(chargePerMassPerRadius)):
       
       for volt in voltPerRadius[i]:
              for k in range(np.size(volt)):
                     volts.append(volt[k])
                     
       for amp in ampPerRadius[i]:
              for k in range(np.size(amp)):
                     amps.append(amp[k])
              
       for campo in magneticFieldPerRadius[i]:
              for k in range(np.size(campo)):
                     mF.append(campo[k])
              
       for razao in chargePerMassPerRadius[i]:
              for k in range(np.size(razao)):
                     CpM.append(razao[k])
              
       dados = {"voltagens": volts,
       "amperagens": amps,
       "campo": mF,
       "razao e/m": CpM}
       
       j = i + 2

       dataFrame = pd.DataFrame(dados)
       dataFrame.to_csv(f'results/dadosRaio{j}cm.dat', index=False, sep=tab)
       
       volts = []
       amps = []
       mF = []
       CpM = []   